[Cats와 Dogs 분류 모델]

In [3]:
import os
import cv2
import zipfile

[1] 파일 압축 해제

In [4]:
ZIP = '../data/dataset/cat_dog.zip'
FOLDER = '../data/dataset/'

[2] 사용자 정의 함수

In [5]:
# 압축 해제
with zipfile.ZipFile(ZIP, 'r') as zip_ref:
	zip_ref.extractall(FOLDER)

In [6]:
#압축 해제된 파일 목록 확인
FOLDER2 = '../data/dataset/cat_dog'
extracted_files = os.listdir(FOLDER2)
extracted_files

['cat_(1).jpg',
 'cat_(10).jpg',
 'cat_(11).jpg',
 'cat_(12).jpg',
 'cat_(13).jpg',
 'cat_(14).jpg',
 'cat_(15).jpg',
 'cat_(16).jpg',
 'cat_(17).jpg',
 'cat_(18).jpg',
 'cat_(19).jpg',
 'cat_(2).jpeg',
 'cat_(20).jpg',
 'cat_(21).jpg',
 'cat_(22).jpg',
 'cat_(23).jpg',
 'cat_(24).jpg',
 'cat_(25).jpg',
 'cat_(26).jpg',
 'cat_(27).jpg',
 'cat_(28).jpg',
 'cat_(29).jpg',
 'cat_(3).jpg',
 'cat_(30).jpg',
 'cat_(31).jpg',
 'cat_(32).jpg',
 'cat_(33).jpg',
 'cat_(34).jpg',
 'cat_(35).jpg',
 'cat_(36).jpg',
 'cat_(37).jpg',
 'cat_(38).jpg',
 'cat_(39).jpg',
 'cat_(4).jpg',
 'cat_(40).jpg',
 'cat_(41).jpg',
 'cat_(42).jpg',
 'cat_(43).jpg',
 'cat_(44).jpg',
 'cat_(45).jpg',
 'cat_(46).jpg',
 'cat_(47).jpg',
 'cat_(48).jpg',
 'cat_(49).jpg',
 'cat_(5).jpg',
 'cat_(50).jpg',
 'cat_(51).jpg',
 'cat_(52).jpg',
 'cat_(53).jpg',
 'cat_(54).jpg',
 'cat_(55).jpg',
 'cat_(56).jpg',
 'cat_(57).jpg',
 'cat_(58).jpg',
 'cat_(59).jpg',
 'cat_(6).jpg',
 'cat_(60).jpg',
 'cat_(61).jpeg',
 'cat_(62).jpg',
 

In [7]:
import numpy as np
import pandas as pd
import subprocess

In [8]:
# 데이터 로드
data = 'cat_dog_dataset.csv'
image = []
labels = []

In [9]:
for root, _, files in os.walk(FOLDER2):
	for file in files:
		if file.lower().endswith((".jpg", ".png")):
			img = os.path.join(root, file)
			label = 0 if "cat" in file.lower() else 1
			image.append(img)
			labels.append(label)

In [10]:
# 데이터프레임 생성 및 저장
df = pd.DataFrame({"image_path": image, "label": labels})
df.to_csv(data, index=False)

In [11]:
# 이미지 데이터 로드 및 전처리
X, y = [], []
for index, row in df.iterrows():
    img_path, label = row["image_path"], row["label"]
    if os.path.exists(img_path):
        img = cv2.imread(img_path)  # 이미지 읽기
        img = cv2.resize(img, (128, 128))  # 크기 조정
        img = img.flatten()  # 1차원 벡터로 변환
        X.append(img)
        y.append(label)
    else:
        print(f"이미지 파일 없음: {img_path}")

In [12]:
# NumPy 배열 변환
if len(X) == 0:
    raise ValueError("이미지 데이터가 로드되지 않았습니다. 파일 경로를 확인하세요.")
X = np.array(X)
y = np.array(y)

모델 생성 및 학습

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random

In [14]:
# 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [15]:
# 로지스틱 회귀 모델 학습
log_reg = make_pipeline(StandardScaler(), LogisticRegression())
log_reg.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [16]:
# 모델 평가
y_pred = log_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"로지스틱 회귀 정확도: {accuracy:.4f}")

로지스틱 회귀 정확도: 0.8214


In [17]:
# 분류 리포트 출력
print("\n분류 리포트:")
print(classification_report(y_test, y_pred, target_names=["Cat", "Dog"]))


분류 리포트:
              precision    recall  f1-score   support

         Cat       0.82      0.75      0.78        12
         Dog       0.82      0.88      0.85        16

    accuracy                           0.82        28
   macro avg       0.82      0.81      0.82        28
weighted avg       0.82      0.82      0.82        28



In [18]:

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_test, y_pred)
print("\n혼동 행렬:")
print(conf_matrix)



혼동 행렬:
[[ 9  3]
 [ 2 14]]


In [19]:
# 랜덤 이미지 예측 실행
if len(X_test) > 0:
    random_index = random.randint(0, len(X_test) - 1)
    sample_image = X_test[random_index].reshape(1, -1)
    true_label = "Cat" if y_test[random_index] == 0 else "Dog"
    predicted_label = "Cat" if log_reg.predict(sample_image)[0] == 0 else "Dog"
    print(f"\n실제 라벨: {true_label}, 예측된 라벨: {predicted_label}")
else:
    print("테스트 데이터가 없습니다. 데이터 로드를 확인하세요.")


실제 라벨: Dog, 예측된 라벨: Dog
